In [3]:
dataset_name = "W19_comb"
df_list = []# [ "BES_Panel" ]

In [27]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

%matplotlib inline

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pickle, os, gc, re

sns.set();
sns.set_palette("colorblind")

from IPython.display import display, display_html, HTML
from IPython.core.debugger import set_trace
# plt.rcParams["axes.grid"] = False

import Jupyter_module_loader
from utility import *
import gaussian_kde

import warnings
warnings.filterwarnings('ignore')

import holoviews as hv
from holoviews import opts

encoding = "ISO-8859-1"

# you should clone this git to a subdirectory called 'BES_analysis_code' (in some directory - I call it BES_analysis - doesn't matter though)
# %matplotlib inline
(BES_code_folder, BES_small_data_files, BES_data_folder,
 BES_output_folder, BES_file_manifest, BES_R_data_files) = setup_directories()

global BES_Panel, BES_numeric, BES_reduced, BES_reduced_with_na, BES_non_numeric
data_subfolder = BES_data_folder + dataset_name + os.sep

(manifest, dataset_filename, dataset_description, dataset_citation,
 dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)

for df in df_list:
    if df=="BES_Panel":
        globals()[df]  = pd.read_pickle(data_subfolder + dataset_filename.replace('.dta','.zip'),compression='zip')
    else:
        globals()[df]  = pd.read_pickle(data_subfolder + df + '.zip',compression='zip' )
        globals()[df].replace(-1,np.nan,inplace=True)
  
(var_type, cat_dictionary, new_old_col_names, old_new_col_names) = get_small_files(data_subfolder, encoding)

# get full set of inferred "cross wave" auth-lib/left-right values and ages
# pan_dataset_allr_values = pd.read_csv(BES_small_data_files + "pan_dataset_allr_values"+".csv")
# pan_dataset_ages = pd.read_pickle(BES_small_data_files + "pan_dataset_ages"+".zip", compression='zip')

Toggle code

NameError: name 'dataset_name' is not defined

Media variables
all these variables reflect the "profile_newspaper_readership_201" yougov variable

note that this is "Which daily newspaper do you read most often?"

you could legitimately read the Express and Mail about as often and oscillate between

also there's a huge editorial (and anecdotally I understand readership) difference between

Mail on Sunday (Remain! - contains weekly TV listings) and the Daily Mail (Leave!)

"profiles_newspaper2W16" is obviously just a "top up" sample

"profile_newspaper" is an amalgamation of all the waves

Full Waves 10, 11, 12, 13, 15

In [3]:
search(BES_Panel,"newspaper")

newspaperReadW15          30842
profile_newspaperW10      30237
profile_newspaperW11      30956
profile_newspaper2W12     34394
profile_newspaperW13      31136
profiles_newspaper2W16    13775
profiles_newspaper2W17    34366
profile_newspaper         56968
dtype: int64

In [ ]:
# they're a bit None heavy (and region matters)

In [4]:
BES_Panel["profile_newspaper"].value_counts()

None                                           17415
The Sun                                         7420
The Daily Mail / The Scottish Daily Mail        7390
The Guardian                                    4461
The Mirror / Daily Record                       4407
Other Newspaper                                 3743
The Daily Telegraph                             2726
The Times                                       2720
Other local daily morning newspaper             2265
The Independent                                 1339
The Express                                     1068
The Herald (Glasgow)                             562
The Daily Star / The Daily Star of Scotland      539
The Scotsman                                     391
The Financial Times                              319
The Western Mail                                 203
Name: profile_newspaper, dtype: int64

In [ ]:
BES Media module variables
infoSource

search(BES_Panel,"infoSource")

During the last seven days, on average how much time (if any) have you spent per day following news about

politics or current affairs from each of these sources?

TV/Paper/Radio/Internet/People

Full waves 4, 5, 6, 7, 8, 12, 13, 15

In [5]:
BES_Panel["infoSourceTVW4"].value_counts()

Less than 1/2 hour      8861
1/2 hour to 1 hour      8585
1 to 2 hours            5334
None, no time at all    4199
More than 2 hours       3505
Don't know              1068
Name: infoSourceTVW4, dtype: int64

In [ ]:
twitterUse: Full Waves 4, 5, 6, 12

Do you use Twitter? Yes/No/DK (mostly No!, ~8000/30+k = Yes)

~8000 people that say Yes then asked

twitterInfo(_1/2/3) Partial Sample Waves 4, 5, 12

In the last 4 weeks, have you read news or found information about the upcoming general election or politics

more generally that was tweeted by any of the following people or organisations? Tick all that apply

twitterInfo_1: Candidates or parties on Twitter

twitterInfo_2: Someone you know personally on Twitter

twitterInfo_3: Other people on Twitter e.g. commentators, journalists, activists

Yes/No/DK/Don't follow politics on twitter (mostly Don't follow politics on twitter)

twitterInfof2f Partial Sample Wave 6
During the election campaign, did you get news about politics and the election through twitter?
Yes/No/DK

In [6]:
BES_Panel["twitterInfof2fW6"].value_counts()

Yes           4505
No            3821
Don't know     114
Name: twitterInfof2fW6, dtype: int64

In [ ]:
fbUse: Full Waves 4, 5, 6, 12

Do you use Facebook? Yes/No/DK (mostly No!, ~20k/30+k = Yes)

~20000 people that say Yes then asked

fbInfo(_1/2/3) Partial Sample Waves 4, 5, 12

In the last 4 weeks, have you read news or found information about the upcoming general election or politics more

generally that was posted on Facebook by any of the following people or organisations? Tick all that apply

fbInfo_1 Candidates or parties on Facebook

fbInfo_2 Someone you know personally on Facebook

fbInfo_3 Other people on Facebook e.g. commentators, journalists, activists

Yes/No/DK/Don't follow politics on facebook (mostly Don't follow politics on facebook - even moreso than twitter)

fbInfof2fW6 Partial Sample Wave 6
During the election campaign, did you get news about politics and the election through Facebook?
Yes/No/DK

In [7]:
BES_Panel["fbInfo_1W4"].value_counts()

Don't follow politics on facebook    14358
No                                    3038
Yes                                   2609
Don't know                            1003
Name: fbInfo_1W4, dtype: int64

In [ ]:
euSources Full Waves 7, 8

Have you heard anything about the EU referendum from any of these during the last seven days?

Please tick all that apply.

euSources_1: Television

euSources_2: Newspaper (including online)

euSources_3: Radio

euSources_4: Internet (not including online newspapers)

euSources_5: Talking to other people

Yes/No/DK

In [ ]:
need to download/open up the open-ended question dataset for
tvWatch/radioList/paperRead/internetRead

urk - ugly human data

What are the most important Internet sources (including online newspapers) you read

for information about politics and the election?

List up to three websites or click next if there are no websites you can think of.

TV/radio/newspaper/internet



In [39]:
df[match(df,"TV",case_sensitive=False).index].stack().value_counts().head(50)

                           586358
__na__                     184623
bbc news                    16733
question time               10075
newsnight                    8514
news                         8054
sky news                     4775
itv news                     3861
daily politics               3441
none                         2878
channel 4 news               2168
this week                    1541
sunday politics              1521
andrew marr                  1494
bbc                          1412
itv                           998
dk                            942
news night                    799
panorama                      663
andrew marr show              647
leaders debate                613
bbc breakfast                 560
bbc1                          490
the news                      488
na                            478
scotland tonight              450
local news                    436
sky                           429
debate                        412
bbc news 24   

In [47]:
df[match(df,"radio",case_sensitive=False).index].stack().value_counts().head(50)

__na__                   237862
none                       5370
today                      4531
news                       2363
radio 4                    2052
pm                         1796
radio 2                    1299
dk                         1071
lbc                         882
na                          856
jeremy vine                 852
world at one                770
bbc news                    720
today programme             660
0                           652
any questions               610
radio 4 news                470
radio 5                     432
5 live                      426
radio 1                     405
radio 5 live                357
bbc radio 4                 342
bbc radio 2                 310
heart                       307
bbc                         307
five live                   307
local radio                 287
radio scotland              280
good morning scotland       266
radio 2 news                230
radio4                      218
question

In [53]:
df[match(df,"paper",case_sensitive=False).index].stack().value_counts().head(50)

__na__              224217
guardian              5768
daily mail            5537
none                  4008
telegraph             3244
times                 2915
sun                   2515
independent           2120
mirror                1893
mail                  1832
i                     1814
metro                 1773
the times             1682
the guardian          1549
the sun               1526
daily telegraph       1289
daily mirror          1195
sunday times           987
observer               805
evening standard       784
daily express          751
express                739
the national           720
dk                     682
na                     674
mail on sunday         597
the independent        594
daily record           466
scotsman               465
0                      441
herald                 389
independant            346
the mirror             338
the herald             338
the telegraph          335
ft                     303
sunday herald          282
t

In [54]:
df[match(df,"internet",case_sensitive=False).index].stack().value_counts().head(50)

__na__                 228761
bbc                      8368
bbc news                 7057
none                     3940
guardian                 3208
facebook                 1815
twitter                  1755
sky news                 1373
telegraph                1196
daily mail               1186
huffington post          1060
dk                        872
yahoo                     836
independent               833
bbc.co.uk                 730
sky                       725
the guardian              723
google                    695
msn                       650
na                        630
wings over scotland       561
0                         492
mail                      354
bbc website               326
mail online               301
times                     292
yougov                    287
google news               285
guido fawkes              272
daily telegraph           242
bella caledonia           227
itv                       215
reddit                    208
bbc news w

In [60]:
sorted(list(df.columns))

['MIIEU_textW15',
 'MIIEU_textW7',
 'MIIEU_textW8',
 'MII_textW1',
 'MII_textW10',
 'MII_textW11',
 'MII_textW12',
 'MII_textW13',
 'MII_textW14',
 'MII_textW15',
 'MII_textW16',
 'MII_textW17',
 'MII_textW18',
 'MII_textW19',
 'MII_textW2',
 'MII_textW3',
 'MII_textW4',
 'MII_textW5',
 'MII_textW6',
 'MII_textW7',
 'MII_textW8',
 'MII_textW9',
 'ReasonForvoteOthW13',
 'ReasonForvoteOthW19',
 'ReasonForvoteOthW6',
 'bestOnMIIOthW1',
 'bestOnMIIOthW10',
 'bestOnMIIOthW11',
 'bestOnMIIOthW12',
 'bestOnMIIOthW13',
 'bestOnMIIOthW14',
 'bestOnMIIOthW15',
 'bestOnMIIOthW16',
 'bestOnMIIOthW17',
 'bestOnMIIOthW18',
 'bestOnMIIOthW19',
 'bestOnMIIOthW2',
 'bestOnMIIOthW3',
 'bestOnMIIOthW4',
 'bestOnMIIOthW5',
 'bestOnMIIOthW6',
 'bestOnMIIOthW7',
 'bestOnMIIOthW8',
 'bestOnMIIOthW9',
 'brexitPrefOthW17',
 'brexitPrefOthW18',
 'brexitPrefOthW19',
 'changeViewDetailW12',
 'changeViewDetailW13',
 'changeViewDetailW18',
 'changeViewDetailW19',
 'contactMPSubjectW4',
 'conwinreasonopenW17',
 'conwinreasonopenW18',
 'euExpLeaveNegW7',
 'euExpLeavePosW7',
 'euExpRemainNegW7',
 'euExpRemainPosW7',
 'euNonVoteReasonsW16',
 'euVoteReasonsW16',
 'euroElectionVoteNonVoterOthW16',
 'euroElectionVoteOthW1',
 'euroElectionVoteOthW16',
 'euroElectionVoteOthW2',
 'euroElectionVoteYoungOthW2',
 'geNonVoteIntentionReasonsW18',
 'gePostVoteReasonsW18',
 'geVoteIntentionReasonsW18',
 'genElecNonVotIndW19',
 'genElecNonVotOthW19',
 'genElecOthCodedTIGW15',
 'genElecOthCodedW15',
 'genElecVoteRetro2017OthW18',
 'generalElectionSqueezeIndW17',
 'generalElectionSqueezeIndW18',
 'generalElectionSqueezeIndW19',
 'generalElectionVoteIndW17',
 'generalElectionVoteIndW18',
 'generalElectionVoteOthTIGW15',
 'generalElectionVoteOthW1',
 'generalElectionVoteOthW10',
 'generalElectionVoteOthW11',
 'generalElectionVoteOthW12',
 'generalElectionVoteOthW14',
 'generalElectionVoteOthW15',
 'generalElectionVoteOthW16',
 'generalElectionVoteOthW17',
 'generalElectionVoteOthW18',
 'generalElectionVoteOthW2',
 'generalElectionVoteOthW3',
 'generalElectionVoteOthW4',
 'generalElectionVoteOthW5',
 'generalElectionVoteOthW6',
 'generalElectionVoteOthW7',
 'generalElectionVoteOthW8',
 'generalElectionVoteOthW9',
 'generalElectionVoteOthwishW6',
 'generalElectionVotePostIndW18',
 'generalElectionVotePostOthW12',
 'generalElectionVotePostOthW5',
 'indContactW17',
 'indContactW18',
 'indContactW19',
 'internet1_1W4',
 'internet1_1W5',
 'internet1_1W8',
 'internet2_1W4',
 'internet2_1W5',
 'internet2_1W8',
 'internet3_1W4',
 'internet3_1W5',
 'internet3_1W8',
 'knowMPOthW1',
 'knowMPOthW2',
 'knowMPOthW3',
 'localElectionVoteOthW12',
 'localElectionVoteOthW14',
 'localElectionVoteOthW16',
 'localElectionVoteOthW2',
 'localElectionVoteOthW6',
 'localElectionVoteOthW7',
 'localElectionVoteOthW8',
 'mpNameW4',
 'othContactW1',
 'othContactW11',
 'othContactW12',
 'othContactW13',
 'othContactW14',
 'othContactW16',
 'othContactW17',
 'othContactW18',
 'othContactW19',
 'othContactW2',
 'othContactW3',
 'othContactW4',
 'othContactW5',
 'othContactW6',
 'othContactW7',
 'othContactW8',
 'othMpPartyW4',
 'othSubjClassW10W11',
 'othSubjClassW12W14',
 'othSubjClassW15',
 'othSubjClassW16',
 'othSubjClassW17',
 'othSubjClassW19',
 'othSubjClassW2_W4W7W9',
 'othersectorW19',
 'paper1_1W4',
 'paper1_1W5',
 'paper1_1W8',
 'paper2_1W4',
 'paper2_1W5',
 'paper2_1W8',
 'paper3_1W4',
 'paper3_1W5',
 'paper3_1W8',
 'partyId4OthW16',
 'partyId5OthW16',
 'partyIdANESOthW16',
 'partyIdBSAOthW16',
 'partyIdCSESOthW16',
 'partyIdOth1TIGW15',
 'partyIdOthW1',
 'partyIdOthW10',
 'partyIdOthW11',
 'partyIdOthW12',
 'partyIdOthW13',
 'partyIdOthW14',
 'partyIdOthW15',
 'partyIdOthW16',
 'partyIdOthW17',
 'partyIdOthW18',
 'partyIdOthW19',
 'partyIdOthW2',
 'partyIdOthW3',
 'partyIdOthW4',
 'partyIdOthW6',
 'partyIdOthW7',
 'partyIdOthW8',
 'partyIdOthW9',
 'partyIdSqueezeOth1TIGW15',
 'partyIdSqueezeOthW1',
 'partyIdSqueezeOthW10',
 'partyIdSqueezeOthW11',
 'partyIdSqueezeOthW12',
 'partyIdSqueezeOthW13',


In [67]:
df[search(df,"q1",case_sensitive=False).index]

,q1_before_yesterdayW18,q1_before_yesterday_newsW18,q1_yesterdayW18,q1_yesterday_newsW18,q1_todayW18,q1_today_newsW18
id,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN
2,__na__,__na__,__na__,__na__,__na__,__na__
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
98447,NaN,NaN,NaN,NaN,NaN,NaN
98448,NaN,NaN,NaN,NaN,NaN,NaN
98449,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
crap_list = ["__na__","none","dk","__dk__","na","0","?",".","don't know","dont know","-","d/k","n/a","o","n",
             "x","dn","no","n.a","a","not sure","1","2","3","4","5","6","7","8","9","non","nil","no tv","no idea",
             "unsure","s","/",","," none","n/c","don't read newspapers","n a","nothing","h","g","any",
             "i don't read newspapers","f","e","all","c","??","z","l","v","???","q","b","i don't know","idk","do not read",
             "pass","various","m","do not know","an","as above","..","not applicable","d"]
crap_regex = ["^the "]
replace_dict = dict(zip(crap_list,[np.nan]*len(crap_list)))


In [84]:
df[match(df,"q1|tv|radio|paper|internet",case_sensitive=False).index].replace(replace_dict).replace(crap_regex,"",regex=True).stack().value_counts().head(400)[-60:]

                   

# pm? no? 1? 2? non? bt? 3? 4? 5?
# pm is a radio station!

# I think anything that's 1 letter is "woops - I want to get out of here" - except maybe "i" (what about 1/2/3 radio?)

qt                                                                                         65
panarama                                                                                   65
201911241835|itv1|itv evening news|news                                                    65
201912091800|bbc1|bbc news at six|news~201912091830|bbc1|regional news and weather|news    64
itv1                                                                                       64
201911201800|bbc1|bbc news at six|news~201911201830|bbc1|regional news and weather|news    64
201911251930|bbc1|the andrew neil interviews|current affairs: politics                     64
pass                                                                                       64
talksport                                                                                  64
ch4 news                                                                                   63
201911181800|bbc1|bbc news|news~201911181830|bbc1|regional n

In [97]:
df[match(df,"tv",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True).stack().value_counts().head(300)[-60:]

euro news                    18
main news                    18
today in parliament          18
welsh news                   18
bbc 6pm news                 18
bbc wales                    18
granada reports              17
bbc 10pm news                17
don't watch                  17
political debate             17
all party debate             17
politics programme           17
bbc ten o'clock news         17
pass                         17
andrew marrs                 17
six o clock news             17
i have no tv                 17
bbc news 10pm                17
weekly politics              16
opposition debate            16
bbc 24 news                  16
bloomberg                    16
bbc24                        16
bbc 6 oclock news            16
itv new                      16
politic show                 16
10 o clock news              16
horizon                      16
unsure                       16
party debates                16
andrew mar show              16
week in 

In [103]:
df[match(df,"radio",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True).stack().value_counts().head(300)[-60:]

,                       12
forth one               12
radio 4 morning news    12
not applicable          12
radio sheffield         12
radio wm                12
radio kent              12
bbc2 news               12
debate                  12
drivetime               12
scotland today          12
good morning            12
1 o'clock news          12
local station           12
morning call            12
classic                 12
one o'clock news        12
daily politics          12
politics today          12
6 o clock news          11
radio newcastle         11
news at ten             11
wave                    11
bbc radio two           11
bbc 5                   11
lbc radio               11
radio humberside        11
taro'r post             11
morning news            11
jeremy vine r2          11
bbc 5live               10
news programmes         10
archers                 10
5 live daily            10
question time extra     10
bbc newcastle           10
radio norfolk           10
x

In [111]:
df[match(df,"paper",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True).stack().value_counts().head(290)[-60:]


derby telegraph         5
argus                   5
indipendant             5
oxford mail             5
herald express          4
eadt                    4
worcester news          4
l                       4
gusrdian                4
huffingdon post         4
v                       4
idependent              4
independence            4
tines                   4
yep                     4
???                     4
dont read newspapers    4
sum                     4
daiy mail               4
local weekly            4
evening gazette         4
mirrror                 4
guaridan                4
daily mirrow            4
daiky mail              4
telegaph                4
 sun                    4
q                       4
guardiam                4
golwg                   4
daliy mail              4
huffington post uk      4
dandy                   4
guardin                 4
b                       4
theguardian             4
daily telegrapgh        4
daily miror             4
telegrath   

In [117]:
df[match(df,"internet",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True).stack().value_counts().head(280)[-60:]

m                          11
week                       11
do not know                11
gov.uk                     11
glasgow herald             11
msn.com                    10
bbc/news                   10
daily post                 10
lbc                        10
an                         10
talk talk                  10
thetimes.co.uk             10
daily mail website         10
daily mash                 10
local newspaper            10
www.dailymail.co.uk        10
conservative website       10
le monde                   10
golwg 360                  10
daily mail on line         10
bt yahoo                   10
msm                        10
conservative               10
uk polling                  9
timesonline                 9
news now                    9
politics                    9
money saving expert         9
huffingdon post             9
labour website              9
nine                        9
plaid cymru                 9
manchester evening news     9
bbc wales 

In [123]:
df[match(df,"tv|radio|paper|internet",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True).stack().value_counts().head(540)[-60:]



not applicable                26
derek bateman                 26
week in politics              26
express and star              26
wales this week               25
..                            25
yahoo.co.uk                   25
bellacaledonia                25
radio five                    25
electoral calculus            25
murnaghan                     25
emmerdale                     25
hard talk                     25
news at six                   25
5live breakfast               25
sunday politics scotland      25
itv news at 10                24
d                             24
news channel                  24
radio 2 jeremy vine           24
party election broadcast      24
good                          24
what the papers say           24
time                          24
question time extra time      24
may2015                       24
welsh news                    24
flipboard                     24
heart news                    24
six o clock news              23
pm questio

In [128]:
df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan).stack().value_counts().head(180)[-60:]

201911171830|channel 4|channel 4 news|news                                                                                           52
201912071720|bbc1|regional news and weather|news                                                                                     52
201911231800|itv1|itv evening news|news~201911231815|itv1|regional news and weather|news                                             51
201912090600|bbc1|breakfast|daytime                                                                                                  51
201911242230|itv1|itv news|news                                                                                                      50
201912091300|bbc1|bbc news at one|daytime                                                                                            50
201911261800|bbc1|bbc news at six|news~201911261830|bbc1|regional news and weather|news                                              50
201912011710|bbc1|regional news and weather|news

In [190]:
df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: len(x.split("~")) if pd.notnull(x) else 0).stack().value_counts()

0    529894
1     25428
2     13861
3      6398
4      3367
5      2155
6      1508
7       336
8         1
dtype: int64

In [139]:
ser = pd.Series( flatten( df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: x.split("~") if pd.notnull(x) else np.nan).stack().values ) )
ser.value_counts()

#.value_counts().head(60)[-60:]

201911172200|bbc1|bbc news|news                                                    589
201911242200|bbc1|bbc news|news                                                    572
201911192000|itv1|johnson v corbyn: the itv debate|documentary: current affairs    567
201912012200|bbc1|bbc news|news                                                    555
201911221900|bbc1|question time leaders special|current affairs: politics          552
                                                                                  ... 
2019112822                                                                           1
201912022100|bbc1|the prince and the epstein scandal - panorama|do                   1
201912091800|itv1|regional news and weath                                            1
201911211830|bbc1|regional news and weather                                          1
201912101900|channel 4|channel 4 n                                                   1
Length: 3384, dtype: int64

In [164]:
sec_no = pd.Series(section_no)
sec_no.value_counts()

4    2353
3     780
2     142
1     109
dtype: int64

In [171]:
# ser.value_counts().apply(lambda x: len(x.index.split("|")))
section_no = [len(x.split("|")) for x in ser.value_counts().index]
ser.value_counts().loc[ser.value_counts().index[sec_no>=2]]

201911172200|bbc1|bbc news|news                                                    589
201911242200|bbc1|bbc news|news                                                    572
201911192000|itv1|johnson v corbyn: the itv debate|documentary: current affairs    567
201912012200|bbc1|bbc news|news                                                    555
201911221900|bbc1|question time leaders special|current affairs: politics          552
                                                                                  ... 
201911241830|skynews|sophy ridge on sunda                                            1
201912022100|bbc1|the prince and the epstein scandal - panorama|do                   1
201912091800|itv1|regional news and weath                                            1
201911211830|bbc1|regional news and weather                                          1
201912101900|channel 4|channel 4 n                                                   1
Length: 3275, dtype: int64

In [187]:
q1_tv_programmes = pd.Series(flatten([x.split("|")[1:] for x in ser.value_counts().loc[ser.value_counts().index[sec_no>=2]].index]))
q1_tv_programmes.value_counts().head(60)#.head(240)[-60:]

news                                                 1490
bbcnews24                                             788
bbc1                                                  760
itv1                                                  629
skynews                                               567
daytime                                               336
bbc news                                              222
regional news and weather                             218
bbc2                                                  169
channel 4                                             131
channel 5                                             119
                                                       88
bbc news at ten                                        81
press preview                                          73
sky news                                               73
the papers                                             70
current affairs: politics                              59
sportsday     

In [ ]:
# q1 column structure
# 0-8 tuples
# tuples (1-4 elements)
# datetime numeric (yearmonthday24hr) | prog | prog | prog

# the string occasionally breaks early
# might be possible to reconstitute

In [ ]:
whole bunch of variables about visiting a party site/sharing party website/fb/twitter stuff -> ignoring

In [5]:
dataset_name = "W19_comb_strings"
data_subfolder = BES_data_folder + dataset_name + os.sep

(manifest, dataset_filename, dataset_description, dataset_citation,
 dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)

In [9]:
data_subfolder

'../BES_analysis_data/W19_comb_strings\\'

In [12]:
df = pd.read_stata(data_subfolder + dataset_filename, encoding =encoding)

TypeError: read_stata() got an unexpected keyword argument 'encoding'

In [13]:
encoding = "ISO-8859-1"

ValueError: Only file formats >= 113 (Stata >= 9) are supported.  Got format 60.  Please report if you think this error is incorrect.

In [1]:
encoding

NameError: name 'encoding' is not defined

In [ ]:
import pyreadstat

dtafile = data_subfolder + dataset_filename
df, meta = pyreadstat.read_dta(dtafile)

In [6]:
data_subfolder + dataset_filename

'../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.2.dta'

In [9]:
df = pd.read_stata('../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.2_newer.dta')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 6: invalid start byte

In [10]:
import statsmodels as sm

from statsmodels.iolib.foreign import StataReader

with open('../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.2_newer.dta', 'rb') as stata_file:
    df = StataReader(stata_file, encoding =encoding)

ValueError: Only file formats >= 113 (Stata >= 9) are supported.  Got format 60.  Please report if you think this error is incorrect.

In [ ]:
import pyreadstat

dtafile = '../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.2_newer.dta'
df, meta = pyreadstat.read_dta(dtafile)

In [23]:
import numpy as np

In [3]:
import pandas as pd
csvfile = '../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.2.csv'
encoding = "ISO-8859-1"
df = pd.read_csv(csvfile,encoding=encoding)

df.rename(columns={'ï»¿id':'id'},inplace=True)

In [11]:
df = df.set_index("id").sort_index()
df

,MII_textW1,bestOnMIIOthW1,generalElectionVoteOthW1,generalElectionVoteOthW2,generalElectionVoteOthW3,generalElectionVoteOthW4,generalElectionVoteOthW5,generalElectionVotePostOthW5,generalElectionVoteOthW6,generalElectionVoteOthwishW6,...,indContactW19,othContactW19,changeViewDetailW19,othSubjClassW19,othersectorW19,othSubjClassW2_W4W7W9,othSubjClassW10W11,othSubjClassW12W14,genElecOthCodedW15,genElecOthCodedTIGW15
id,,,,,,,,,,,,,,,,,,,,,
1,This awful coalition,__NA__,__NA__,__NA__,,__NA__,__NA__,__NA__,__NA__,__NA__,...,,,,,,,,,,
2,Terrorism,__NA__,__NA__,,__NA__,__NA__,__NA__,__NA__,__NA__,__NA__,...,__NA__,,__NA__,,__NA__,,,,,
3,__NA__,__NA__,__NA__,__NA__,__NA__,__NA__,__NA__,__NA__,__NA__,__NA__,...,__NA__,,__NA__,,__NA__,,,,,
4,economy,__NA__,__NA__,__NA__,,,,,,,...,,,,,,,,,,
5,__NA__,__NA__,__NA__,__NA__,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98447,,,,,,,,,,,...,,,,,,,,,,
98448,,,,,,,,,,,...,,,,,,,,,,
98449,,,,,,,,,,,...,__NA__,,__NA__,,__NA__,,,,,


In [43]:
df = df.apply(lambda x: x.str.lower().replace(" ",np.nan).astype('category'))

In [26]:
df.to_pickle('../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.2.zip', compression='zip')

In [46]:
df

,MII_textW1,bestOnMIIOthW1,generalElectionVoteOthW1,generalElectionVoteOthW2,generalElectionVoteOthW3,generalElectionVoteOthW4,generalElectionVoteOthW5,generalElectionVotePostOthW5,generalElectionVoteOthW6,generalElectionVoteOthwishW6,...,indContactW19,othContactW19,changeViewDetailW19,othSubjClassW19,othersectorW19,othSubjClassW2_W4W7W9,othSubjClassW10W11,othSubjClassW12W14,genElecOthCodedW15,genElecOthCodedTIGW15
id,,,,,,,,,,,,,,,,,,,,,
1,this awful coalition,__na__,__na__,__na__,NaN,__na__,__na__,__na__,__na__,__na__,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,terrorism,__na__,__na__,NaN,__na__,__na__,__na__,__na__,__na__,__na__,...,__na__,NaN,__na__,NaN,__na__,NaN,NaN,NaN,NaN,NaN
3,__na__,__na__,__na__,__na__,__na__,__na__,__na__,__na__,__na__,__na__,...,__na__,NaN,__na__,NaN,__na__,NaN,NaN,NaN,NaN,NaN
4,economy,__na__,__na__,__na__,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,__na__,__na__,__na__,__na__,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,__na__,NaN,__na__,NaN,__na__,NaN,NaN,NaN,NaN,NaN


In [18]:
vc = df.stack().str.lower().value_counts()
vc.head(10)
# "" most common
# "__NA__" second most
# "Brexit" third
# "None" fourth

               20807298
__na__          6018837
brexit           147146
immigration       65562
none              47432
economy           35543
nhs               24900
bbc news          24604
the economy       13601
terrorism         12736
dtype: int64

In [20]:
vc.head(50)

                    20807298
__na__               6018837
brexit                147146
immigration            65562
none                   47432
economy                35543
nhs                    24900
bbc news               24604
the economy            13601
terrorism              12736
news                   10603
question time          10289
bbc                    10245
guardian                8994
newsnight               8538
climate change          7201
daily mail              6731
independent             6294
sky news                6177
__dk__                  5770
poverty                 5450
leaving the eu          5167
inequality              5142
eu referendum           4774
housing                 4652
today                   4587
telegraph               4445
dk                      4340
itv news                3966
crime                   3817
eu                      3545
imigration              3469
daily politics          3457
europe                  3352
times         

In [19]:
vc.shape

(254871,)